In [36]:
import os
import numpy as np
import pandas as pd
import hickle
import cv2
import PIL
import time

from PIL import Image

# http://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script
import sys
sys.path.append('C:/Anaconda/Lib/site-packages')



# Set DATA directory
DATA_DIR = 'C:/Users/thoma/Documents/00GitHub/rbc_cnn/'
CSV_DIR = DATA_DIR + 'csv/'

# Set IMAGE directory: Keep off of GitHub
IMG_DIR = 'C:/Users/thoma/Documents/00GitHub/00_LOCAL_ONLY/00RbcCNN_Sln_Images/'

# read in cell labels from Cell Label Tool 
classes = pd.read_csv(DATA_DIR + 'dataset.csv', index_col=0, parse_dates=True)

# get rid of reject cells
classes = classes[classes.label != 'Reject']
classes = classes[classes.label != 'Abnormal']

# counts the number of cells in each calss
#print classes['label'].value_counts()

def csv_to_dataFrame(path):
    
    # output csv
    #fout=open(outputFile,"a")
    i = 0 

    for root, dirs, files in os.walk(path):
        frame = pd.DataFrame()
        list_ = []
        for file_ in files:
            csv_path = os.path.join(root, file_)
            df = pd.read_csv(csv_path,index_col=0, header=0)
            list_.append(df)
        frame = pd.concat(list_)
    frame = frame.reset_index(drop=True)
    return frame

ImportError: No module named cv2

In [4]:
df = pd.DataFrame(np.random.randint(0,10,size=(10, 4)), columns=list('ABCD'))
df

,A,B,C,D
0,3,2,7,4
1,0,1,7,9
2,9,3,0,4
3,0,0,4,7
4,0,2,4,9
5,0,5,1,2
6,2,2,9,7
7,7,3,6,9
8,5,0,7,5
9,8,1,0,4


In [5]:
df = df.drop(df.index[3])
df

,A,B,C,D
0,3,2,7,4
1,0,1,7,9
2,9,3,0,4
4,0,2,4,9
5,0,5,1,2
6,2,2,9,7
7,7,3,6,9
8,5,0,7,5
9,8,1,0,4


In [33]:
df

,A,B,C,D
0,3,2,7,4
1,0,1,7,9
2,9,3,0,4
3,0,2,4,9
4,5,0,7,5
5,8,1,0,4


In [32]:
for n in range(len(df)):
    # select one cell to compare to rest of dataset
    cell = df.index[n]
    if n == 4:
        df = df.drop(df.index[n])
        df = df.reset_index(drop=True)
    print cell, n

0 0
1 1
2 2
3 3
4 4
5 5


IndexError: index 6 is out of bounds for axis 0 with size 6

In [34]:
def parse_dataFrame(df):
    "remove double labeled cells"
    i = 0
    for1 = 0
    for2 = 0
    missedByY = 0
    overlap_list = []

    # sort df by x value
    df = df.sort_values(by=['x'], ascending=[True])
    df = df.reset_index(drop=True)

    for n in range(len(df)-1):

        if n >= len(df):
            break

        # select one cell to compare to rest of dataset
        cell = df.ix[n]
        
        start = n - 30
        stop = n + 30

        if 0 > n - 30:
            start = 0
        else:
            start = start

        if len(df) < stop:
            stop = len(df)
        else:
            stop = stop

        for s in range(start, stop):
            second_cell = df.ix[s]
            olc = overlap_cell_check(df, cell, second_cell)
            if olc != None:
                overlap_list.append(olc)
                df = df.drop(df.index[n])
                df = df.drop(df.index[s])
                df = df.reset_index(drop=True)
                i += 1
                print "{} overlapping cells".format(i)
            else:
                continue

    overlap_df = pd.DataFrame(overlap_list)

In [35]:
df, overlap_df = parse_dataFrame(df)

KeyError: 'x'